In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45895
2,Huelva,Confirmados PDIA 14 días,296
3,Huelva,Tasa PDIA 14 días,"57,680690609349725"
4,Huelva,Confirmados PDIA 7 días,152
5,Huelva,Tasa PDIA 7 dias,"29,619814096693105"
6,Huelva,Total Confirmados,46098
7,Huelva,Curados,44940
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45895.0


/tmp/ipykernel_9540/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12623.0


/tmp/ipykernel_9540/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_9540/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_9540/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_9540/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5327 personas en los últimos 7 días 

Un positivo PCR cada 2820 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45895.0,152.0,296.0,513170.0,29.619814,57.680691,44.0
Huelva-Costa,27148.0,88.0,161.0,289548.0,30.392197,55.603907,23.0
Sierra de Huelva-Andévalo Central,4063.0,28.0,42.0,67391.0,41.548575,62.322862,14.0
Huelva (capital),12623.0,27.0,51.0,143837.0,18.771248,35.456802,9.0
Campofrío,35.0,13.0,13.0,713.0,1823.281907,1823.281907,8.0
Cartaya,2079.0,36.0,62.0,20083.0,179.256087,308.718817,8.0
Condado-Campiña,14258.0,36.0,92.0,156231.0,23.042802,58.887161,7.0
Ayamonte,2098.0,8.0,11.0,21104.0,37.907506,52.122820,3.0
Lepe,3036.0,9.0,13.0,27880.0,32.281205,46.628407,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,35.0,13.0,13.0,713.0,1823.281907,1823.281907,8.0
Cartaya,2079.0,36.0,62.0,20083.0,179.256087,308.718817,8.0
Zalamea la Real,122.0,6.0,8.0,3054.0,196.463654,261.951539,2.0
Palos de la Frontera,1054.0,6.0,16.0,11742.0,51.098620,136.262988,2.0
Moguer,2001.0,14.0,27.0,21867.0,64.023414,123.473728,1.0
Palma del Condado (La),1430.0,6.0,11.0,10801.0,55.550412,101.842422,2.0
Aroche,131.0,2.0,3.0,3054.0,65.487885,98.231827,0.0
Minas de Riotinto,181.0,3.0,3.0,3812.0,78.698846,78.698846,1.0
Valverde del Camino,859.0,1.0,9.0,12750.0,7.843137,70.588235,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,859.0,1.0,9.0,12750.0,7.843137,70.588235,0.0,0.111111
Almonte,2310.0,3.0,10.0,24507.0,12.241400,40.804668,0.0,0.300000
Aljaraque,1633.0,1.0,3.0,21474.0,4.656794,13.970383,0.0,0.333333
Aracena,672.0,1.0,3.0,8255.0,12.113870,36.341611,0.0,0.333333
Isla Cristina,3146.0,5.0,15.0,21393.0,23.372131,70.116393,1.0,0.333333
Palos de la Frontera,1054.0,6.0,16.0,11742.0,51.098620,136.262988,2.0,0.375000
Bollullos Par del Condado,1295.0,3.0,8.0,14387.0,20.852158,55.605755,1.0,0.375000
Condado-Campiña,14258.0,36.0,92.0,156231.0,23.042802,58.887161,7.0,0.391304
Punta Umbría,1276.0,2.0,5.0,15355.0,13.025073,32.562683,0.0,0.400000
